In [1]:
import cv2

import numpy as np

from sklearn.metrics import pairwise

In [2]:
## creating global varibels

background = None

accumulated_weight = 0.3

## setting my ROI

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [3]:
## creating a func to find the background avg value

def calc_accum_avg (frame, accumulated_weight):
    
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        return None
    ## if the "backbackground" as value this following line will update it.
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [4]:
## Segmenting the hand in ROI - creating a func using thresholding to grab the hand segment 

def segment(frame, threshold_min=25):
    
    # calc the diff betweene the backbackground & the past in frame
    
    diff = cv2.absdiff(background.astype('uint8'), frame) # getting the absolut diff img
    
    # i only need the threshold, so i will throw away the first item in the tuple with an underscore _
    _ , thresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)
    
    # Grabing the external contours from the img 
    image, contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # quick check if i found any contours
    if len(contours) == 0:
        return None
    # assuming the largest contour in roi, is the hand
    else:
        
        hand_segment = max(contours, key=cv2.contourArea)
        
        return(thresholded, hand_segment)


## Creating a Counting fingers func

In [5]:
# After calculating the external contour of the hand. Now i can calculate the Convex Hull of the hand segmented.
## A Convex Hull may be visualized as the shape enclosed by a rubber band stretched around the subset.
def count_fingers(thresholded, hand_segment):
    
    conv_hull = cv2.convexHull(hand_segment)
    
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottum = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    # the center of the hand should be the half way from top to botteum and left and right
    cX = (left[0] + right[0]) // 2  # index[0] for the x cordinate
    cY = (top[1] + bottum[1]) // 2  # index[1] for the y cordinate
    
    # calculating the euclidean distance between the center and top/left/bottum/right
    distance = pairwise.euclidean_distances([(cX,cY)], Y=[left, right, top, bottum])[0]
    
    # calculating the max distane later to use for the radius and circumfrence
    max_distance = distance.max()
    
    radius = int(0.9*max_distance)
    
    circumfrence = 2*np.pi*radius
    
    # creating ROI for the circule the thresholded img with only the x,y cordinat -- I dont need the color chanel
    circular_roi = np.zeros(thresholded.shape[:2], dtype='uint8')

    # drawing the circule
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10) ## I acualy wont be viewing this just calculating the circle
    
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)
    
    # grabing all the contours in the circular_roi
    
    image, contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    # counting the fingers
    
    count = 0
    
    for cnt in contours:
        
        (x,y,h,w) = cv2.boundingRect(cnt)
        
        out_of_weist = ((cY + (cY*0.25)) > (y+h)) # checking if some of the contours are of the wrist: True=finger
        limit_points = ((circumfrence*0.25) > cnt.shape[0]) # avoiding background noises: True=finger
        
        if out_of_weist and limit_points :
            count += 1
            
    return count
        
        
        
    
    
    

## BRINGING IT ALL TOGETHER

In [6]:
cam = cv2.VideoCapture(0)

num_frame = 0

while True:
    
    ret, frame = cam.read()
    
     # flip the frame so that it is not the mirror view
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()
    
    # defined my ROI
    roi = frame_copy[roi_top:roi_bottom, roi_right:roi_left]
    
    # getting a grayscale of the ROI img
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    # applying blure
    gray = cv2.GaussianBlur(gray, (7,7), 0)
    
    if num_frame < 60:  ## caculating the background
        
        calc_accum_avg(gray, accumulated_weight)
        
        if num_frame <=59:
            
            cv2.putText(frame_copy, 'WAIT ! CALCULATING BACKGROUND', (200,300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Finger Count', frame_copy)
            
    else:
        
        # entering the hand & using the segment func
        hand = segment(gray)
        
        if hand is not None:
            
            thresholded, hand_segment = hand
            
            # highlighting the hand in the real img
            cv2.drawContours(frame_copy, [hand_segment+(roi_right, roi_top)], -1, (255,0,0), 5)
            
            # counting the fingers
            fingers = count_fingers(thresholded, hand_segment)
            
            cv2.putText(frame_copy, str(fingers), (70,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255), 2)
            
            # showing also the thresholded img
            cv2.imshow('thresholded', thresholded)
            
    # drawing the ROI rectangle
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)
    
    num_frame += 1
    
    cv2.imshow('Finger count', frame_copy)
    
    k = cv2.waitKey(1) & 0Xff
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()